In [1]:
import tensorflow as tf
import numpy as np
import scipy.io
from pyDOE import lhs
import math

D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or '1ty

In [2]:
RandomSeed = 9
np.random.seed(RandomSeed)
tf.set_random_seed(RandomSeed)

In [3]:
class PtPINNsss:
    # Initialize the class
    def __init__(self, x, t, u, lb, ub, lbp,ubp, layers,result_weights_values,result_biases_values):
        
        X = np.concatenate([x, t], 1)

        self.X = X
        
        self.x = X[:,0:1]
        self.t = X[:,1:2]
        
        self.u = u      
        self.lb = lb
        self.ub = ub
        self.hsadasjd=0
        self.ubp = ubp
        self.lbp = lbp        
        # Initialize NNs
        self.layers = layers
        self.hh=[]
        self.hh1=[]      
        
        self.weights, self.biases = self.initialize_NN(layers)
        
        # Calculate the values and convert them to float32
        value_float64 = 0.9
        self.b = tf.Variable(value_float64, dtype=tf.float64)
        
        value_float64=  1.0/(math.cosh(5*0.5)-1)       
        
        self.a = tf.constant(value_float64, dtype=tf.float64)     
        
        
        
        self.weights_values = result_weights_values        
        self.biases_values = result_biases_values    

        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))    

        self.x_f_tf = tf.placeholder(tf.float64, shape=[None, self.x.shape[1]])
        self.t_f_tf = tf.placeholder(tf.float64, shape=[None, self.t.shape[1]])
        
        self.x_lb_tf = tf.placeholder(tf.float64, shape=[None, self.x.shape[1]])
        self.t_b_tf = tf.placeholder(tf.float64, shape=[None, self.t.shape[1]])
        self.x_ub_tf = tf.placeholder(tf.float64, shape=[None, self.x.shape[1]])
        
        self.x_tf = tf.placeholder(tf.float64, shape=[None, self.x.shape[1]])
        self.t_tf = tf.placeholder(tf.float64, shape=[None, self.t.shape[1]])
        
        self.u_tf = tf.placeholder(tf.float64, shape=[None, self.u.shape[1]])

        self.u_pred, _ ,self.uuuuuu = self.net_AC(self.x_tf, self.t_tf)
        self.u_lb_pred, self.ux_lb_pred,_ = self.net_AC(self.x_lb_tf, self.t_b_tf)
        self.u_ub_pred, self.ux_ub_pred,_ = self.net_AC(self.x_ub_tf, self.t_b_tf)

        self.f_pred = self.net_f(self.x_f_tf, self.t_f_tf)
        

        
        self.lossS = tf.reduce_mean(tf.square(self.u_tf - self.u_pred))                                              
        self.lossB = tf.reduce_mean(tf.square(self.u_lb_pred - self.u_ub_pred)) + tf.reduce_mean(tf.square(self.ux_lb_pred - self.ux_ub_pred))
        self.lossfu =  tf.reduce_mean(tf.square(self.f_pred))    
        

        self.optimizer_Adam = tf.train.AdamOptimizer()

        self.loss  =   self.lossB + self.lossfu
        
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)      

        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
    
        init = tf.global_variables_initializer()
        self.sess.run(init)
        self.save = tf.train.Saver(max_to_keep=1)
              
    def initialize_NN(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = tf.Variable(tf.zeros([layers[l], layers[l+1]], dtype=tf.float64), dtype=tf.float64)
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float64), dtype=tf.float64)
            weights.append(W)
            biases.append(b)        
        return weights, biases
    
    
    def initialize_NN2(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float64), dtype=tf.float64)
            weights.append(W)
            biases.append(b)        
        return weights, biases      
    
        
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev, dtype=tf.float64))
    
    def custom_function(self,t):
        ab= self.b     
        condition2 = tf.less_equal(t, 0.8)
        condition3 = tf.less_equal(t, ab)
        condition4 = tf.less_equal(ab, 1.0) 
        BB=5*(t-0.8)
        B=1/(ab-0.8)*(t-0.8)
        
        return tf.where(condition2, tf.zeros_like(t), tf.where(condition4,tf.where(condition3, -2*B**3+3*B**2,tf.zeros_like(t)+1),-2*BB**3+3*BB**2))
    
 

    def input_encoding(self, t, x):    
        H=tf.concat([t,1+0*tf.cos(math.pi*x)],1)
        H=tf.concat([H,tf.cos(math.pi*x)],1)
        H=tf.concat([H,tf.sin(math.pi*x)],1) 
        H=tf.concat([H,tf.cos(2*math.pi*x)],1)
        H=tf.concat([H,tf.sin(2*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(3*math.pi*x)],1)
        H=tf.concat([H,tf.sin(3*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(4*math.pi*x)],1)
        H=tf.concat([H,tf.sin(4*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(5*math.pi*x)],1)
        H=tf.concat([H,tf.sin(5*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(6*math.pi*x)],1)
        H=tf.concat([H,tf.sin(6*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(7*math.pi*x)],1)
        H=tf.concat([H,tf.sin(7*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(8*math.pi*x)],1)
        H=tf.concat([H,tf.sin(8*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(9*math.pi*x)],1)
        H=tf.concat([H,tf.sin(9*math.pi*x)],1)         
        H=tf.concat([H,tf.cos(10*math.pi*x)],1)
        H=tf.concat([H,tf.sin(10*math.pi*x)],1)           
        
        return H


    def neural_net(self, x,t, weights, biases,weights_values,biases_values):
        X = tf.concat([x,t],1)
        tt=5*t
        num_layers = len(weights) + 1
        H = self.input_encoding(tt, x)
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            W1 = weights_values[l]
            b1 = biases_values[l]   
            H0 = tf.add(tf.matmul(H, W), b)
            H1 = tf.add(tf.matmul(H, W1), b1)
            
            B = self.custom_function(t)   
            H0 = H0*B              
            
            H  = tf.add(H0,H1)
            H =  tf.tanh(H)
        
        W = weights[-1]
        b = biases[-1]
        W1 = weights_values[-1]
        b1 = biases_values[-1]  
        
        H0 = tf.add(tf.matmul(H, W), b)
        
        B = self.custom_function(t)   
        H0 = H0*B            
        
        H1 = tf.add(tf.matmul(H, W1), b1) 
        H  = tf.add(H0,H1)      
        Y = H
        return Y
    
    def net_AC(self, x, t):
        u = self.neural_net(x,t, self.weights, self.biases,self.weights_values, self.biases_values)
        u_x = tf.gradients(u, x)[0]
        u_t = tf.gradients(u, t)[0]

        return u, u_x, u_t
          
    

    def net_f(self, x, t):
        u, u_x, u_t = self.net_AC(x, t)
        
        u_xx = tf.gradients(u_x, x)[0]
        
        f_u = u_t - 0.0001*u_xx+5*u**3-5*u
        
        return f_u
    
    def callback(self, loss, a,b,lossfu, lossS, lossB):
        sss=self.hsadasjd
        if sss%1000==0:
            print('Loss: %.6e, Lossfu: %.3e, LossS: %.3e, LossB: %.3e ' % (loss, lossfu, lossS, lossB))
        sss=sss+1
        self.hsadasjd=sss 
        self.hh.append(a)      
        self.hh1.append(b)        
    def train(self, nIter, Nf, Nn, Nb):

        X_train = self.lbp + (self.ubp-self.lbp)*lhs(2, Nf)
        self.xtrain_f = X_train[:,0:1]
        self.ttrain_f = X_train[:,1:2] 
        
        X_lb_train = self.lbp + [0,self.ubp[1]-self.lbp[1]]*lhs(2, Nb)
        self.xtrain_lb = X_lb_train[:,0:1]
        self.ttrain_b = X_lb_train[:,1:2]
        
        X_ub_train = [self.ubp[0],self.lbp[1]] + [0,self.ubp[1]-self.lbp[1]]*lhs(2, Nb)
        self.xtrain_ub = X_ub_train[:,0:1]
        
        tf_dict = {self.x_tf: self.x, self.t_tf: self.t, self.u_tf: self.u,
                   self.x_lb_tf: self.xtrain_lb, self.t_b_tf: self.ttrain_b, 
                   self.x_ub_tf: self.xtrain_ub, 
                   self.x_f_tf: self.xtrain_f, self.t_f_tf: self.ttrain_f}

        for it in range(nIter):
            loss_value = self.sess.run(self.loss, tf_dict)
            lossfu = self.sess.run(self.lossfu, tf_dict)
            lossS = self.sess.run(self.lossS, tf_dict)
            lossB = self.sess.run(self.lossB, tf_dict)
            self.sess.run(self.train_op_Adam, tf_dict)
                
            ab= self.sess.run(self.a)
            self.hh.append(ab)       
            abc= self.sess.run(self.b)
            self.hh1.append(abc)    

            print(abc)                    
            if it % 1000 == 0:
                print('It: %d, Loss: %.6e, Lossfu: %.3e, LossS: %.3e, LossB: %.3e' % (it, loss_value, lossfu, lossS, lossB))
                        
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, method = 'L-BFGS-B', options = {'maxiter': 50000,'maxfun': 50000,'maxcor': 50,'maxls': 50,'ftol' : 1.0 * np.finfo(float).eps})                                                                                                         
        self.optimizer.minimize(self.sess, feed_dict = tf_dict, fetches = [self.loss,self.a,self.b,  self.lossfu, self.lossS, self.lossB], loss_callback = self.callback)        
                                    
    
    def predict(self, x, t):
        
        tf_dict = {self.x_tf: x, self.t_tf: t}
        u_star = self.sess.run(self.u_pred, tf_dict)
        u_starssss = self.sess.run(self.uuuuuu, tf_dict)        
        return u_star,u_starssss

    def saver(self, string):
        self.save.save(self.sess, 'ckpt/'+string)
        
        
    def sssss(self):
        return self.hh,  self.hh1           
        
        
    def restore(self):
        model_file = tf.train.latest_checkpoint('ckpt/')
        self.save.restore(self.sess, model_file)

In [4]:
import pickle

In [5]:
with open('bcweights.pkl', 'rb') as f:
    weights_values = pickle.load(f)

In [6]:
with open('bcweights1.pkl', 'rb') as f:
    biases_values = pickle.load(f)

In [7]:
with open('2bcweights.pkl', 'rb') as f:
    weights_values2 = pickle.load(f)
with open('2bcweights1.pkl', 'rb') as f:
    biases_values2 = pickle.load(f)

In [8]:
with open('3bcweights.pkl', 'rb') as f:
    weights_values3 = pickle.load(f)
with open('3bcweights1.pkl', 'rb') as f:
    biases_values3 = pickle.load(f)

In [9]:
with open('4bcweights.pkl', 'rb') as f:
    weights_values4 = pickle.load(f)
with open('4bcweights1.pkl', 'rb') as f:
    biases_values4 = pickle.load(f)

In [10]:
wwweights_values = [x + y for x, y in zip(weights_values, weights_values2)]
rrrresult_biases_values = [x + y for x, y in zip(biases_values, biases_values2)]

In [11]:
grfgsfresult_weights_values = [x + y for x, y in zip(wwweights_values, weights_values3)]
resufsdfsdlt_biases_values = [x + y for x, y in zip(rrrresult_biases_values, biases_values3)]

In [12]:
result_weights_values = [x + y for x, y in zip(grfgsfresult_weights_values, weights_values4)]
result_biases_values = [x + y for x, y in zip(resufsdfsdlt_biases_values, biases_values4)]

In [13]:
if __name__ == "__main__": 
           
    
    # Doman bounds
    lb = np.array([-1.0, 0])
    ub = np.array([1.0, 0.2])

    # Pre-training interval
    lbp = np.array([-1.0, 0.8])    
    ubp = np.array([1.0,  1.0])


    
    layers = [22, 100, 100, 100, 100, 1]
    
    data = scipy.io.loadmat('AC.mat')
    
    t = data['tt'].flatten()[:,None]
    x = data['x'].flatten()[:,None]
    Exact = data['uu']
    
    X, T = np.meshgrid(x,t)
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact.T.flatten()[:,None]

    def IC(x):
        u = x**2*np.cos(np.pi*x)
        return u

    N0 = 1200
    x=np.linspace(-1,1,N0).flatten()[:,None]  
    X0 =x
    T0 = np.full((N0,1), lb[1])
    U0 = IC(X0)
      
    model2 = PtPINNsss(X0, T0, U0, lb, ub,lbp, ubp, layers,result_weights_values,result_biases_values)                        
    model2.train(5000, 4000, 1200, 1200)  
    model2.saver('testmodel.ckpt')
    


    u_pred ,_= model2.predict(X_star[:,0:1],X_star[:,1:2])

    erroru = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
    erroru1 = np.linalg.norm(u_star-u_pred,1)/len(X_star)
    erroruinf = np.linalg.norm(u_star-u_pred,np.inf)
    
    print('randorm seed: %d' % (RandomSeed))
    print('Training error in pre-training interval:(%.2f,%.2f)' % (lb[1], ubp[1]) ) 
    print('Error2 u: %e' % (erroru))
    print('Error1 u: %e' % (erroru1))
    print('Errorf u: %e' % (erroruinf))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
0.9
It: 0, Loss: 4.425520e-02, Lossfu: 4.426e-02, LossS: 1.154e-01, LossB: 1.900e-29
0.9007441343310497
0.9013878861022658
0.9020856133313147
0.9027394846665078
0.9033456407699942
0.903958265271349
0.9045882162988174
0.9052084649076689
0.9057845939075252
0.906311602014665
0.9068128727133168
0.9072996866053098
0.9077638952527836
0.9082058503356226
0.9086374717270514
0.9090657120379624
0.9094870896598363
0.909894687021599
0.9102827353843289
0.9106468917291353
0.9109870448984145
0.9113068905358841
0.9116114589092722
0.9119036729386244
0.9121829635369342
0.912447142796655
0.912695918427892
0.9129312913432255
0.913153806016418
0.9133616121176871
0.9135539622550042
0.9137332008202952
0.9139025196812194
0.9140629345697332
0.9142133025442273
0.9143526056365442
0.914481650884346
0.9146023084892816
0.9147159945932934
0.9148232411164677
0.9149241615569282
0.9150189371209287
0.91

0.9186593201507524
0.918666253227475
0.9186731992259086
0.9186801571712399
0.9186871261847447
0.9186941054756019
0.9187010943317232
0.918708092112021
0.9187150979294388
0.9187221112970841
0.9187291317766906
0.9187361589731969
0.9187431925317174
0.9187502321330335
0.918757277490026
0.9187643283444467
0.9187713844640064
0.9187784456397361
0.9187855116924455
0.918792582452633
0.9187996577677845
0.9188067375006331
0.918813821347825
0.9188209092155478
0.9188280010194346
0.9188350966835819
0.9188421961396127
0.9188492993258158
0.9188564061863844
0.918863516670769
0.918870630733143
0.9188777483325455
0.9188848694312253
0.9188919939947819
0.9188991227593847
0.9189062556239862
0.918913392496904
0.9189205439411067
0.9189277089211636
0.9189348865014141
0.9189420758367436
0.9189492761636386
0.9189564867927734
0.9189637071006899
0.9189709365239035
0.9189781745528941
0.9189854207272918
0.9189926746299403
0.9189999359367188
0.9190072043003011
0.9190144795835072
0.9190217614822738
0.9190290497207628
0

0.9223830006183624
0.9223922084506133
0.9224014233448756
0.9224106440904593
0.9224198695955614
0.9224290988753072
0.9224383310416963
0.9224475652936344
0.922456800908857
0.9224660370944486
0.9224752732778809
0.9224845092473968
0.9224937445077758
0.9225029786127602
0.9225122111600513
0.9225214417866193
0.9225306701653124
0.9225399007450675
0.9225491327841131
0.9225583656136294
0.9225675986304224
0.9225768322911188
0.9225860660043923
0.9225952992370026
0.9226045315081091
0.9226137623843057
0.922622991474518
0.9226322184263285
0.9226414429220272
0.9226506646750351
0.9226598829450612
0.9226690975463013
0.922678308311273
0.9226875150891792
0.9226967177441299
0.9227059161533483
0.9227151119437843
0.9227243048377676
0.922733494585009
0.922742680959843
0.922751863758672
0.9227610455300709
0.9227702266080055
0.9227794065059969
0.922788584785973
0.9227977610527182
0.9228069349497336
0.9228161061555453
0.922825274380311
0.9228344393630873
0.9228436008686707
0.9228527631913036
0.9228619256885318
0

0.9302421459006575
0.9302487097670508
0.9302552684337655
0.9302617911156666
0.9302683270334714
0.9302748543663257
0.9302813581382868
0.9302878852654792
0.930294400623036
0.9303009029790759
0.9303074316727186
0.9303139506542455
0.9303204685442689
0.9303270080076761
0.9303335400544976
0.9303400825880008
0.9303466409706225
0.9303531944356754
0.9303597651262254
0.9303663551515471
0.9303729541069921
0.9303795828157921
0.9303862234072436
0.9303928737338966
0.9303995509059173
0.9304062364438186
0.9304129350699955
0.9304196550779079
0.9304263807787883
0.9304331219346055
0.9304398786389839
0.9304466410538699
0.9304534201228039
0.9304602091637643
0.9304670053596745
0.9304738187015502
0.930480639044897
0.930487468595711
0.9304943133361704
0.9305011636272899
0.9305080252022488
0.9305148991146667
0.9305217787455803
0.9305286709840008
0.9305355729127165
0.9305424819973118
0.9305494059457784
0.930556339141778
0.93056328805296
0.9305702546014777
0.9305772326946131
0.930584225399919
0.9305912319284125


0.9394208869903807
0.9394874558850493
0.9395498157392838
0.9396064414329316
0.9396592952594667
0.9397088437239308
0.939754010947674
0.9397961021543657
0.9398356238793938
0.9398719352916513
0.9399056944322576
0.9399373986096234
0.9399667579883507
0.9399939901738987
0.9400195297281775
0.9400433417597346
0.9400654592141587
0.940086189603774
0.9401056154437777
0.9401236636043359
0.9401405201425656
0.9401564022211246
0.9401712385574529
0.9401850742370719
0.9401981982944514
0.9402105790614143
0.940222157716419
0.9402332087416241
0.9402437217008086
0.9402535919533785
0.9402630728311078
0.9402721764159632
0.940280771046811
0.9402890648005968
0.9402970942387193
0.9403047317084228
0.9403121207425771
0.9403193030875047
0.9403261948528241
0.9403329066136474
0.9403394651940185
0.9403458160090503
0.9403520372728811
0.9403581409848961
0.9403641026518915
0.940369971655802
0.9403757401763773
0.9403814139643305
0.9403870319664728
0.94039256482781
0.9403980241134522
0.9404034477177567
0.9404088053012359


...

0.9651099791217861
It: 3000, Loss: 5.944877e-05, Lossfu: 5.945e-05, LossS: 1.154e-01, LossB: 1.440e-29
0.9651139283291537
0.9651178830418944
0.9651218428056506
0.9651258072108083
0.9651297758880382
0.9651337485042697
0.9651377247585944
0.9651417043796572
0.9651456871229814
0.9651496727676403
0.9651536611142424
0.9651576519829236
0.9651616452113972
0.9651656406530996
0.9651696381754968
0.9651736376589565
0.9651776389954904
0.9651816422803624
0.9651856474059289
0.9651896542749931
0.9651936627995654
0.9651976729001065
0.9652016845045639
0.9652056975477493
0.9652097123436282
0.9652137288014497
0.9652177468391697
0.9652217663825484
0.9652257873643512
0.9652298097237143
0.9652338334054448
0.9652378583594894
0.9652418845404962
0.9652459119072394
0.9652499404222556
0.9652539700514867
0.9652580007639391
0.9652620325314263
0.9652660653281836
0.96527009913072
0.9652741338344992
0.9652781694281012
0.9652822059009558
0.9652862432432655
0.965290281445936
0.9652943205004931
0.965298360398978
0.965302

0.9833497231812731
0.9833524844887631
0.983355190310253
0.9833577743131595
0.9833602770420494
0.9833627196830588
0.9833650511952734
0.9833673727579401
0.9833696188504994
0.9833717893247601
0.9833739675087705
0.9833760617613653
0.9833781484396313
0.9833802032069765
0.9833822002632808
0.9833842095674562
0.9833861661630938
0.9833881022787283
0.983390028687376
0.9833919141055828
0.9833937883316407
0.9833956402877826
0.9833974580388118
0.9833992706516965
0.983401056948199
0.983402814821101
0.9834045733420941
0.9834062964816944
0.9834080157315171
0.9834097231322642
0.9834114033226923
0.9834130934587013
0.9834147594423122
0.9834164234558741
0.9834180858944944
0.9834197321999261
0.9834213857436631
0.983423029350895
0.9834246681605426
0.9834263094028365
0.9834279428448982
0.9834295750476623
0.9834312073776058
0.9834328303986289
0.98343445716316
0.9834360785193624
0.9834376938216629
0.9834393140654546
0.9834409241436299
0.9834425381307618
0.9834441488360787
0.9834457559789821
0.983447368508413
0

0.9913453051193114
0.9913464471933693
0.991347590805169
0.9913487359459529
0.9913498826264744
0.9913510308292018
0.9913521805645479
0.9913533318195228
0.9913544845984875
0.9913556388922264
0.9913567947039583
0.9913579520218778
0.99135911085473
0.9913602711840456
0.9913614330215685
0.9913625963494146
0.9913637611751634
0.9913649274893489
0.9913660952879911
0.9913672645730781
0.9913684353313156
0.9913696075711841
0.9913707812761511
0.9913719564541362
0.9913731330922967
0.991374311834724
0.9913754926100163
0.9913766753625464
0.9913778600339628
0.9913790465798169
0.9913802349493257
0.9913814251089841
0.9913826170141948
0.9913838106390369
0.9913850059459577
0.99138620291249
0.991387401509687
0.9913886017160396
0.9913898035101377
0.9913910068715803
0.9913922117841493
0.9913934182300053
0.991394626195175
0.991395835665413
0.9913970466277988
0.9913982590710609
0.9913994729837539
0.9914006883560391
0.9914019051788735
0.9914031234422853
0.9914043431402704
0.9914055642622126
0.9914067868045789
0.

1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040

1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040914039
1.0035356040

In [21]:
    t = data['tt'].flatten()[:,None][160:201] 
    x = data['x'].flatten()[:,None]
    Exact = data['uu'][:,160:201] 
    
    X, T = np.meshgrid(x,t)
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact.T.flatten()[:,None]
    u_pred,_ = model2.predict(X_star[:,0:1],X_star[:,1:2])

    erroru = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
    erroru1 = np.linalg.norm(u_star-u_pred,1)/len(X_star)
    erroruinf = np.linalg.norm(u_star-u_pred,np.inf)
    
    print('randorm seed: %d' % (RandomSeed))
    print('Training error in pre-training interval:(%.2f,%.2f)' % (lb[1], ubp[1]) ) 
    print('Error2 u: %e' % (erroru))
    print('Error1 u: %e' % (erroru1))
    print('Errorf u: %e' % (erroruinf))

randorm seed: 9
Training error in pre-training interval:(0.00,1.00)
Error2 u: 7.557159e-04
Error1 u: 2.572506e-04
Errorf u: 5.876011e-03


In [22]:
    scipy.io.savemat("solution5.mat", {'u': u_pred})

In [23]:
    t = data['tt'].flatten()[:,None][161:201] 
    x = data['x'].flatten()[:,None]
    Exact = data['uu'][:,161:201] 
    
    X, T = np.meshgrid(x,t)
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    u_star = Exact.T.flatten()[:,None]
    u_pred,_ = model2.predict(X_star[:,0:1],X_star[:,1:2])

    erroru = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
    erroru1 = np.linalg.norm(u_star-u_pred,1)/len(X_star)
    erroruinf = np.linalg.norm(u_star-u_pred,np.inf)
    
    print('randorm seed: %d' % (RandomSeed))
    print('Training error in pre-training interval:(%.2f,%.2f)' % (lb[1], ubp[1]) ) 
    print('Error2 u: %e' % (erroru))
    print('Error1 u: %e' % (erroru1))
    print('Errorf u: %e' % (erroruinf))

randorm seed: 9
Training error in pre-training interval:(0.00,1.00)
Error2 u: 7.582690e-04
Error1 u: 2.574920e-04
Errorf u: 5.876011e-03


In [24]:
    scipy.io.savemat("solution55.mat", {'u': u_pred})

In [15]:
weights_values2 = model2.sess.run(model2.weights)
biases_values2 = model2.sess.run(model2.biases)

In [16]:
import pickle
with open('5bcweights.pkl', 'wb') as f:
    pickle.dump(weights_values2, f)
with open('5bcweights1.pkl', 'wb') as f:
    pickle.dump(biases_values2, f)